#Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

warnings.filterwarnings('ignore')
%matplotlib inline

Using TensorFlow backend.


#Loading Data

Here we loaded the dataset filtered by filial number. Also i gave a unique id starting with 1 for every client because the id followed no rules before and was messing with the size of the neural network.

In [4]:
dataset = pd.read_csv('/content/filt.csv')

In [5]:
dataset.head()


,NOME_PRODUTO,QUANTIDADE,COD_CLIENTE,COD_PRODUTO
0,SMARTPHONE SAMSUNG GAL J4 CORE 16GB COBRE,9.0,1,1
1,SUPORTE UNIV. BEDIN BU-25,8.0,2,2
2,SUPORTE UNIV. BEDIN BU-25,7.0,3,2
3,ANTENA 1.70 MT MULTIPONTO CENTURY,7.0,4,3
4,RECEPTOR MIDIA BOX HDTV B3,6.0,5,4


In [6]:
dataset.shape

(9521, 4)

#Split train and test

80% train 20% test

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

In [8]:
train.head()

,NOME_PRODUTO,QUANTIDADE,COD_CLIENTE,COD_PRODUTO
6753,SMARTPHONE LG K11 PLUS DB AZUL,1.0,5258,205
3018,SMARTPHONE SAMSUNG GALAXY J4 PLUS 32GB PRETO,1.0,2429,71
5224,"TELEVISOR SMART 32"" UN32J4290 SAMSUNG",1.0,4134,50
7934,CAIXA AMPLIFICADA CM-09 CONNECT DJ CONTROLLER,1.0,6152,16
9264,SMARTPHONE SAMSUNG GAL J2 CORE 16GB PRETO,1.0,7124,25


In [9]:
test.head()

,NOME_PRODUTO,QUANTIDADE,COD_CLIENTE,COD_PRODUTO
8668,"TELEVISOR 32"" 32S6500S HD LED SEMP TCL",1.0,11,79
8391,SMARTPHONE SAMSUNG GAL A10 A105 32GB PRETO,1.0,6496,40
3879,PEN DRIVE 8GB PD587 TWIST PRETO,1.0,3100,33
5165,SMARTPHONE LG K8 PLUS PLATINUM,1.0,4093,113
7847,SMARTPHONE SAMSUNG GAL J2 CORE 16GB PRATA,1.0,6083,21


In [6]:
n_users = len(dataset.COD_CLIENTE.unique())
n_users

7305

In [7]:
n_prod = len(dataset.COD_PRODUTO.unique())
n_prod

312

# Creating Neural Network
Neural Networks proved there effectivness for almost every machine learning problem as of now and they also perform exceptionally well for recommendation systems.

In [8]:
# creating book embedding path
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_prod+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

# concatenate features
conc = Concatenate()([book_vec, user_vec])

# add fully-connected-layers
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

# Create model and compile it
model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')

In [9]:
from keras.models import load_model

if os.path.exists('regression_model2.h5'):
    model2 = load_model('regression_model2.h5')
else:
    history = model2.fit([train.COD_CLIENTE, train.COD_PRODUTO], train.QUANTIDADE, epochs=30, verbose=1)
    model2.save('regression_model2.h5')
    plt.plot(history.history['loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Training Error")

showing the loss in validation

In [10]:
model2.evaluate([test.COD_CLIENTE, test.COD_PRODUTO], test.QUANTIDADE)

1905/1905 [==============================] - 0s 38us/step


2.5784282769743854

# Making Recommendations

In [46]:

def making_recom(user_data, prod_data, n):
  #Get a np.array with user id and a numpy array with product saw by him and return a dataframe with 5 recommendations
  #user_data = numpy array with a client id number
  #prod_data = numpy array with one or more products that said client saw and want recommendation
  #n  = number of recommendations
  #return a dataframe with n recommendations

  predictions = model2.predict([user_data, prod_data])
  predictions = np.array([a[0] for a in predictions])
  recommend_prod_ids = (-predictions).argsort()[:10]
  products = pd.read_csv('produtos.csv')
  
  reco = products[products['COD_PRODUTO'].isin(recommend_prod_ids)]
  
  return reco

In [53]:
#Creating user id
user = np.array([1 for i in range(len(book_data))])
#user = np.array([1])
products = pd.read_csv('produtos.csv')
user_prod = products[products['COD_PRODUTO'].isin(user)]
print(user_prod)
prod_data = np.array(list(set(dataset.COD_PRODUTO)))
prod_data[:5]

                                    NOME_PRODUTO  COD_PRODUTO
0  SMARTPHONE SAMSUNG GAL J4 CORE 16GB COBRE                1


array([1, 2, 3, 4, 5])

In [54]:
making_recom(user, prod_data)

                                     NOME_PRODUTO  COD_PRODUTO
6   SMARTPHONE LG K9 TV LM-X210BMW INDIGO                    7
12  DVD D-15 KARAOKE MONDIAL                                13
41  TELEVISOR SMART 50" UHD BT UN50RU7100 SAMSUNG           42
97  TELEVISOR SMART 32 PTV32N87SA LED PHILCO                98


In [ ]:
len(book_data)